In [3]:
import os
import shutil

# Define paths
source_root = "/home/sala/data/Soil types"  # Root folder containing multiple subfolders with images
target_folder = "/home/sala/data/soil_images"  # Folder to store renamed images

# Create target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Supported image extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

# Recursively find all image files in subfolders
image_files = []
for root, _, files in os.walk(source_root):
    for file in files:
        if os.path.splitext(file)[1].lower() in image_extensions:
            image_files.append(os.path.join(root, file))

# Rename and move images to the target folder
for i, image_path in enumerate(image_files, start=1):
    # Generate new filename (e.g., soil_0001.jpg)
    new_filename = f"soil_{i:04d}{os.path.splitext(image_path)[1]}"
    
    # Full path for the target
    target_path = os.path.join(target_folder, new_filename)
    
    # Move and rename the file
    shutil.move(image_path, target_path)
    print(f"Renamed and moved: {image_path} -> {target_path}")

print("All images have been renamed and saved in the 'soil_images' folder.")

Renamed and moved: /home/sala/data/Soil types/Peat Soil/26.jpg -> /home/sala/data/soil_images/soil_0001.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/25.jpg -> /home/sala/data/soil_images/soil_0002.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/13.jpg -> /home/sala/data/soil_images/soil_0003.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/5.jpg -> /home/sala/data/soil_images/soil_0004.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/14.jpg -> /home/sala/data/soil_images/soil_0005.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/24.jpg -> /home/sala/data/soil_images/soil_0006.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/4.jpg -> /home/sala/data/soil_images/soil_0007.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/17.jpg -> /home/sala/data/soil_images/soil_0008.jpg
Renamed and moved: /home/sala/data/Soil types/Peat Soil/10.jpg -> /home/sala/data/soil_images/soil_0009.jpg
Renamed and moved: /home/sala/